<a href="https://colab.research.google.com/github/mir-abir-hossain/ml-projects/blob/main/Land%20use%20classification/project_27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### preliminary

In [ ]:
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from skimage.feature import hog
from skimage.color import rgb2gray

%matplotlib inline

In [ ]:
os.chdir('/content/drive/MyDrive/project_27')

In [ ]:
os.listdir()

['imagepointer.csv', 'color_images.csv', 'project_27.ipynb', 'Images']

In [ ]:
df = pd.read_csv('imagepointer.csv')

In [ ]:
df.iloc[0]['file_no'], df.iloc[0]['label']

('agricultural00', 'agricultural')

### feature extraction

In [ ]:
def get_image(row_id, root=os.path.join(os.getcwd(), 'Images')):
    """
    Converts an image number into the file path where the image is located, 
    opens the image, and returns the image as a numpy array.
    """
    directory = f"{row_id['label']}"
    filename = f"{row_id['file_no']}.tif"
    file_path = os.path.join(os.path.join(root, directory), filename)  
    img = Image.open(file_path)
    if img.size!=(256, 256):
      img = img.resize((256, 256))
    return np.array(img)

In [ ]:
def create_features(img):
    # flatten three channel color image
    color_features = img.flatten()
    # convert image to grayscale
    gray_image = rgb2gray(img)
    # get HOG features from grayscale image
    hog_features = hog(gray_image, block_norm='L2-Hys', pixels_per_cell=(16, 16), cells_per_block=(3, 3))
    # combine color and hog features into a single array
    flat_features = np.hstack((color_features, hog_features))
    return flat_features

In [ ]:
def create_feature_matrix(label_dataframe):
    features_list = []
    
    for i in label_dataframe.index:
        # load image
        img = get_image(label_dataframe.loc[i])
        # get features for image
        image_features = create_features(img)
        features_list.append(image_features)
        
    # convert list of arrays into a matrix
    feature_matrix = np.array(features_list, dtype='float16')
    return feature_matrix

In [ ]:
feature_matrix = create_feature_matrix(df)

In [ ]:
feature_matrix.dtype

dtype('float16')

In [ ]:
feature_matrix.nbytes

892432800

### Normalize and train test split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# split out evaluation sets (x_eval and y_eval)
x_interim, x_eval, y_interim, y_eval = train_test_split(feature_matrix,
                                           df['label'].values,
                                           test_size=0.2,
                                           random_state=52)

# split remaining data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x_interim,
                                                    y_interim,
                                                    test_size=0.4,
                                                    random_state=52)

# examine number of samples in train, test, and validation sets
print('x_train shape:', x_train.shape)
print(x_train[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(x_eval.shape[0], 'eval samples')

x_train shape: (1008, 212484)
[1.430e+02 1.350e+02 1.380e+02 ... 4.856e-03 2.559e-02 1.307e-01] train samples
672 test samples
420 eval samples


In [ ]:
# get shape of our training features
print('Training features matrix shape is: ', x_train.shape)

# define standard scaler
ss = StandardScaler()

# fit the scaler and transform the training features
train_stand = ss.fit_transform(x_train)

# transform the test features
test_stand = ss.transform(x_test)

# look at the new shape of the standardized feature matrices
print('Standardized training features matrix shape is: ', train_stand.shape)
print('Standardized test features matrix shape is: ', test_stand.shape)

Training features matrix shape is:  (1008, 212484)
Standardized training features matrix shape is:  (1008, 212484)
Standardized test features matrix shape is:  (672, 212484)


### PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# Instantiate a PCA object with 350 components
pca = PCA(n_components=450)
pca.fit(train_stand)

# use fit_transform on our standardized training features
X_train = pca.transform(train_stand)

# use transform on our standardized test features
X_test = pca.transform(test_stand)

# look at new shape
print('Training features matrix is: ', X_train.shape)
print('Test features matrix is: ', X_test.shape)

Training features matrix is:  (1008, 450)
Test features matrix is:  (672, 450)


In [ ]:
aaaa = pca.explained_variance_ratio_
sum = 0
count = 0
for i in aaaa:
  sum += i
  count += 1
  print(count, sum)

1 0.246096910394099
2 0.2744647508211904
3 0.2999866371879615
4 0.32052424829169573
5 0.33663300005047153
6 0.3520891980273444
7 0.3658595445899129
8 0.37781932483381214
9 0.38915766795454704
10 0.3996344658670916
11 0.40891067834486494
12 0.41769519446032166
13 0.4254439144061641
14 0.43248209966505036
15 0.43939317417798046
16 0.44604636503734835
17 0.45184840368505375
18 0.45751652561897577
19 0.46299577933732594
20 0.4683271354287207
21 0.47357412755890094
22 0.478393564076823
23 0.4830093019658064
24 0.48741890791123904
25 0.491669596393943
26 0.4958797202304078
27 0.4999654417109805
28 0.5038906338128047
29 0.5077234456163527
30 0.5115273729834645
31 0.5150966762843413
32 0.5186216067614586
33 0.5221096113400503
34 0.5254139141347904
35 0.5287033185141078
36 0.5318884383479953
37 0.5349674760204831
38 0.5379476194119973
39 0.5408847428664518
40 0.5438137685745152
41 0.546717979452789
42 0.549413950013983
43 0.5520513972502853
44 0.5546545218957356
45 0.557219124108062
46 0.559744

### model with svm

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
# define support vector classifier
svm = SVC(kernel='rbf', probability=True, random_state=42)

# fit model
# ... YOUR CODE FOR TASK 10 ...
svm.fit(X_train, y_train)

# generate predictions
y_pred = svm.predict(X_test)

# calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Model accuracy is: ', accuracy)

Model accuracy is:  0.36755952380952384


### CNN

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((256, 256)),
     transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))]
)

In [ ]:
dataset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/project_27/Images', transform=transform)

In [ ]:
type(dataset)

torchvision.datasets.folder.ImageFolder

In [ ]:
interm_indices, test_indices = train_test_split(np.arange(len(dataset.targets)),
                                                test_size=0.1,
                                                shuffle=True, 
                                                stratify=dataset.targets)

In [ ]:
interm_target = [dataset.targets[i] for i in interm_indices]

In [ ]:
train_indices, val_indices = train_test_split(interm_indices,
                                                test_size=0.1,
                                                shuffle=True, 
                                                stratify=interm_target)

In [ ]:
final_train_target = [dataset.targets[i] for i in train_indices]

In [ ]:
len([i for i in range(len(final_train_target)) if final_train_target[i]==20])

81

In [ ]:
train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

In [ ]:
len(val_dataset)

189

In [ ]:
train_dataset

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32)

In [ ]:
for i, data in enumerate(val_loader, 0):
  input, label = data
  print(input.shape) 
  break

torch.Size([32, 3, 256, 256])


In [ ]:
class LeNet5(nn.Module):

    def __init__(self):
        super().__init__()
        # 3 input image channel (RGB), 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# obtain the pretrained model
modelvgg16 = torchvision.models.vgg16(pretrained=True)

# freeze the params
for param in modelvgg16.parameters():
    param.requires_grad = False

# replace with your classifier
num_classes = 21
modelvgg16.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [ ]:
modelvgg16 = torchvision.models.vgg16(pretrained=True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [ ]:
print(modelvgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
modelvgg16.classifier[6] = nn.Linear(in_features=4096, out_features=21, bias=True)

In [ ]:
print(modelvgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
modelvgg16.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.SGD(modelvgg16.parameters(), lr=0.001, momentum=0.9)

In [ ]:
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
import copy

In [ ]:
for epoch in range(10):
    start = time.perf_counter()
    modelvgg16.train()
    running_loss = 0.0
    correct_pred = 0
    for index, data in enumerate(train_loader):
        image, label = data
        image = image.to(device)
        label = label.to(device)
        y_pred = modelvgg16(image)
        
        _, pred = torch.max(y_pred, 1)
        correct_pred += (pred == label).sum()
        
        loss = criterion(y_pred, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
                
        running_loss += float(loss.item())
    end = time.perf_counter()
    print('epoch {}/{}\tTrain loss: {:.4f}\tTrain accuracy: {:.2f}%'.
          format(epoch + 1, 10, running_loss / (index + 1), correct_pred.item() / (32 * (index + 1)) * 100))
    print('Time: {:.2f}s'.format(end - start))
print('Finished training!')

epoch 1/10	Train loss: 2.0647	Train accuracy: 46.06%
Time: 397.63s
epoch 2/10	Train loss: 0.8944	Train accuracy: 82.70%
Time: 17.14s
epoch 3/10	Train loss: 0.6364	Train accuracy: 86.17%
Time: 17.11s
epoch 4/10	Train loss: 0.5205	Train accuracy: 88.02%
Time: 16.67s
epoch 5/10	Train loss: 0.4446	Train accuracy: 89.06%
Time: 16.50s
epoch 6/10	Train loss: 0.4002	Train accuracy: 89.76%
Time: 16.67s
epoch 7/10	Train loss: 0.3470	Train accuracy: 91.67%
Time: 16.83s
epoch 8/10	Train loss: 0.3283	Train accuracy: 91.72%
Time: 16.84s
epoch 9/10	Train loss: 0.3077	Train accuracy: 92.30%
Time: 16.71s
epoch 10/10	Train loss: 0.2884	Train accuracy: 92.25%
Time: 16.66s
Finished training!


In [ ]:
os.getcwd()

'/content/drive/MyDrive/project_27'

In [ ]:
torch.save(modelvgg16.state_dict(), 'checkpoint.pth')

In [ ]:
loader_vgg16 = torch.load('checkpoint.pth')

In [ ]:
modelvgg16 = torchvision.models.vgg16(pretrained=False)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
modelvgg16.classifier[6] = nn.Linear(in_features=4096, out_features=21, bias=True)

In [ ]:
modelvgg16.load_state_dict(torch.load('checkpoint.pth'))

<All keys matched successfully>

In [ ]:
modelvgg16.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

#### sample image test

In [ ]:
imgg = Image.open('/content/drive/MyDrive/project_27/Images/buildings/buildings01.tif')

In [ ]:
imgg = transform(imgg)

In [ ]:
imgg = imgg.unsqueeze(0)

In [ ]:
output1 = modelvgg16(imgg)

In [ ]:
_, predicted = torch.max(output1.data, 1)

In [ ]:
print(predicted)

NameError: ignored

### Test Data

In [ ]:
len(test_dataset), len(val_dataset)

(210, 189)

In [ ]:
test_loss = 0.0
correct_pred = 0
for _, data in enumerate(val_loader):
    image, label = data
    image = image.to(device)
    label = label.to(device)
    y_pred = modelvgg16(image)

    _, pred = torch.max(y_pred, 1)
    correct_pred += (pred == label).sum()
    
    loss = criterion(y_pred, label)
    test_loss += float(loss.item())
print('Test loss: {:.4f}\tTest accuracy: {:.2f}%'.format(test_loss / 32, correct_pred.item() / 189 * 100))

Test loss: 0.0394	Test accuracy: 96.30%


In [ ]:
test_loss = 0.0
correct_pred = 0
for _, data in enumerate(train_loader):
    image, label = data
    image = image.to(device)
    label = label.to(device)
    y_pred = modelvgg16(image)

    _, pred = torch.max(y_pred, 1)
    correct_pred += (pred == label).sum()
    
    loss = criterion(y_pred, label)
    test_loss += float(loss.item())
print('Test loss: {:.4f}\tTest accuracy: {:.2f}%'.format(test_loss / 32, correct_pred.item() / 1701 * 100))

Test loss: 0.4147	Test accuracy: 96.12%


In [ ]:
test_loss = 0.0
correct_pred = 0
for _, data in enumerate(test_loader):
    image, label = data
    image = image.to(device)
    label = label.to(device)
    y_pred = modelvgg16(image)

    _, pred = torch.max(y_pred, 1)
    correct_pred += (pred == label).sum()
    
    loss = criterion(y_pred, label)
    test_loss += float(loss.item())
print('Test loss: {:.4f}\tTest accuracy: {:.2f}%'.format(test_loss / 32, correct_pred.item() / 210 * 100))

Test loss: 0.0502	Test accuracy: 96.19%


In [ ]:
y_pred

NameError: ignored

In [ ]:
len(test_loader.dataset)

210

In [ ]:
for _, data in enumerate(test_loader):
  print(_)

0
1
2
3
4
5
6


### Extra

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            running_loss = 0.0
            running_corrects = 0
            # Iterate over data.
            for inputs, labels in train_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()
            epoch_loss = running_loss
            epoch_acc = running_corrects.double()
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            # deep copy the model
        print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ftvgg16 = train_model(modelvgg16, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 2790.4353 Acc: 911.0000
val Loss: 446.1682 Acc: 1570.0000

Epoch 1/24
----------
train Loss: 572.5435 Acc: 1506.0000
val Loss: 194.7869 Acc: 1636.0000

Epoch 2/24
----------
train Loss: 280.4991 Acc: 1607.0000
val Loss: 64.2181 Acc: 1680.0000

Epoch 3/24
----------
train Loss: 119.7822 Acc: 1661.0000
val Loss: 75.9691 Acc: 1676.0000

Epoch 4/24
----------
train Loss: 90.7320 Acc: 1669.0000
val Loss: 13.5576 Acc: 1699.0000

Epoch 5/24
----------
train Loss: 75.3580 Acc: 1673.0000
val Loss: 17.8438 Acc: 1695.0000

Epoch 6/24
----------
train Loss: 38.4419 Acc: 1693.0000
val Loss: 18.0091 Acc: 1697.0000

Epoch 7/24
----------
train Loss: 36.6299 Acc: 1688.0000
val Loss: 1.8360 Acc: 1701.0000

Epoch 8/24
----------
train Loss: 9.7979 Acc: 1699.0000
val Loss: 1.1328 Acc: 1701.0000

Epoch 9/24
----------
train Loss: 8.7415 Acc: 1698.0000
val Loss: 0.8259 Acc: 1701.0000

Epoch 10/24
----------
train Loss: 5.7160 Acc: 1701.0000
val Loss: 0.7452 Acc: 1701.0000


In [ ]:
model_ftvgg16.parameters

<bound method Module.parameters of VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size